# 方程式の解

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

方程式とは未知数を含む等式のことである<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)。方程式を解くとは、その等式を満たすような未知数を求めるということである。方程式にはさまざまな種類のものがある。たとえば、

<!-- textlint-enable -->

$$
x^2 - x - 6 = 0 ,
\tag{1}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

のような（未知数$x$に対する）代数方程式であったり、

<!-- textlint-enable -->

$$
e^x = 3 x ,
\tag{2}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

のような超越方程式であったり、あるいは、

<!-- textlint-enable -->

$$
\begin{cases}
2 x + 3 y = 6 , \\
\phantom{2} x + \phantom{3} y = 1 ,
\end{cases}
\tag{3}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

のような（未知数$x$と$y$に対する）連立2元1次方程式であったりする。今回は、式<!-- eqref -->(1)や<!-- eqref -->(2)のように未知数が1つである方程式を考えることにしよう<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)。その未知数は$x$とする。右辺が$0$でない場合はそれを左辺に移項できるので、一般に、

<!-- textlint-enable -->

$$
f(x) = 0 ,
\tag{4}
$$

と書くことができる。式<!-- eqref -->(1)のような2次方程式であれば解の公式を使うことができるが、5次以上の代数方程式には（代数的に）解ける一般の公式は存在しないことが知られている。また、式<!-- eqref -->(2)のような超越方程式も、一般には解析的に解くことができない。よって、式<!-- eqref -->(4)を満たすような未知数$x$を数値的に求めよう、というのがここでの目的である。

方程式によっては解が複数個ある場合もあるが、今回はそのうちの**実数解を1つだけ**どうにかして求めることができればよい、ということにしよう。一般には、方程式の複数の解をすべて求めるということは難しい問題である。

**注意**：実際の研究では、よく整備されたライブラリがすでにあれば、それを利用すべきである。今回の場合で言うと、たとえば[SciPy](https://www.scipy.org/scipylib/)というライブラリには[scipy.optimize.root_scalar](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html) という関数が存在する。ただし、今回習うような基本的なアルゴリズムを知っておくとライブラリの中で何をやっているかを理解しやすくなり、予想していた解がうまく得られないなど困難にぶつかったとき原因を特定しやすくなるだろう。

## 今回のねらい

- ニュートン法による求根の方法を理解する。
- 二分法による求根の方法を理解する。
- 数値計算のアルゴリズムにはそれぞれ得手不得手があることを実感する。

## 関数の概形を描く

「彼（敵）を知り己を知れば百戦殆うからず」という孫子の言葉がある。まずは、今回の「敵」である式<!-- eqref -->(4)の$f(x)$のような関数の概形を知るために、グラフを簡単に描く関数を定義してみよう。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


def make_plot(f, x1, x2):
    # 関数f(x)に対し、x1からx2までの範囲のグラフを表示する関数
    xpoints = np.linspace(x1, x2, 200)
    ypoints = np.vectorize(f)(xpoints)
    fig, ax = plt.subplots()
    ax.plot(xpoints, ypoints)
    ax.axhline(color="red")  # x軸（y=0）を赤い線で描く
    ax.grid()
    plt.show()

ここでは、引数に与えられた関数`f`に対し[`numpy.vectorize`関数](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html)を適用することで「ベクトル化」してから（その結果として得られる関数に）`xpoints`を渡すことで`ypoints`を得ている。また、[`Axes`クラス](https://matplotlib.org/stable/api/axes_api.html#the-axes-class)の[`axhline`メソッド](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.axhline.html)を$x$軸を描くために使用している。

上で定義した`make_plot`関数の使用例として、$x^2 - x - 6 = 0$の左辺を$-4 \le x \le 4$の範囲で描いてみよう。

In [ ]:
def f(x):
    return x**2 - x - 6


make_plot(f, -4, 4)

$x^2 - x - 6 = 0$の解が$x = -2$付近と$x = 3$付近にあることが分かるだろう（実際に$x = -2$と$x = 3$が解である）。

## ラムダ式

上の例では、`def`文で関数`f`を定義して、それを`make_plot`関数に引数として渡している。これでも特に問題はないが、わざわざ（中身はたった1行の短い`return`文であるような）関数を定義し、それに`f`という名前を与えている。あとで関数`f`を何回も再利用するのでなければ、これは少々仰々しいとも思える。

Pythonでは、このようなちょっとした関数を表すための[ラムダ式](https://docs.python.org/ja/3/tutorial/controlflow.html?highlight=%E3%83%A9%E3%83%A0%E3%83%80#lambda-expressions)というものが用意されており、名前のない小さな関数を作ることができる。たとえば、`lambda x: x ** 2`と書くと、引数`x`を取り、`x ** 2`を返す関数となる。上の例を、ラムダ式で作った関数を直接`make_plot`に渡すことで書くと、次のようになる。

In [ ]:
make_plot(lambda x: x**2 - x - 6, -4, 4)

## 練習問題

(1) $e^x - 3x = 0$ について、グラフを描くことによって、解がどの辺りにあるか検討せよ（解は2つある）。$e^x$については、[mathモジュール](https://docs.python.org/ja/3/library/math.html)の[exp関数](https://docs.python.org/ja/3/library/math.html#math.exp)、あるいは[NumPy](https://numpy.org/)の[`numpy.exp`関数](https://numpy.org/doc/stable/reference/generated/numpy.exp.html)を使えばよい。

(2) $x^5 + x - 3 = 0$について、グラフを描くによって、解がどの辺りにあるか検討せよ（解は1つである）。

## ニュートン法

![ニュートン法](images/newton.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図1. ニュートン法

<!-- textlint-enable -->

いま、関数$f(x)$とその導関数$f'(x)$が分かっているものとして、$f(x)=0$の解を数値的に求めることを考えよう。ある$x$の値$x_0$を適当に取り、曲線 $f(x)$上の点$(x_0, f(x_0))$を通る接線を引く（図1）。ただし、$f'(x) \ne 0$と仮定する。その接線と$x$軸との交点の$x$座標を$x_1$とすると、$x_1$は $x_0$よりも$f(x) = 0$を満たす$x$の値に（少なくとも図1の場合には）近づいていることが分かるだろう。$x_1$の値は次のように求めることができる。

$$
x_1 = x_0 - \frac{f(x_0)}{f'(x_0)} .
\tag{5}
$$

次に、曲線$f(x)$上の点$(x_1, f(x_1))$を通る接線を引き、その接線と$x$軸との交点の$x$座標を$x_2$とする。このとき、$x_2$は$x_1$よりもさらに解に近い値となっている。このような操作を繰り返すことにすると、次の漸化式を得る。

$$
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} .
\tag{6}
$$

式<!-- eqref -->(6)の反復によって近似解を求める方法をニュートン法（Newton's method）と呼ぶ。

In [ ]:
def newton(f, fprime, x0, n=100, verbose=False):
    # 方程式f(x)=0の解を、x0から出発してニュートン法で求めて返す関数。
    # fprime(x)は微分を返すとする。
    #
    # n: ニュートン法の反復回数
    # verbose: Trueなら途中の収束状況を表示
    x = x0

    for i in range(n):
        if verbose:
            print(f"i={i}, x={x}")

        # 漸化式
        x = x - f(x) / fprime(x)

    return x

簡単のため、ここでは収束判定やエラー処理などを省いている<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3)。

上で定義した `newton` 関数を $x^2 - x - 6 = 0$ に対して用いてみよう。まずは$x_0=4$を初期値とする。

In [ ]:
newton(lambda x: x**2 - x - 6, lambda x: 2 * x - 1, 4)

複数の解がある場合、初期値を変えると、収束する先の解も変わる。今度は$x_0=-4$としてみよう。

In [ ]:
newton(lambda x: x**2 - x - 6, lambda x: 2 * x - 1, -4)

<!-- textlint-disable ja-engineering-paper/prh,ja-technical-writing/max-ten -->

ニュートン法は、初期値$x_0$が十分に解に近ければ、非常に速く収束することが知られている（2次収束）。また、1変数実関数の求解だけでなく、複素関数や多変数関数の求解へと拡張も可能である。しかし、導関数$f'(x)$が必要であり<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4)、また$f'(x) = 0$となる点があると、ゼロ除算によるエラーが起きたり、うまく収束しないこともある。

<!-- textlint-enable -->

## 練習問題

(3) `newton`関数は`verbose=True`をオプション引数に与えると、収束の状況を逐一表示する。上の例（$x^2 - x - 6 = 0$）に対して、初期値$x_0$をいろいろと変えながら、収束の様子を確かめよ。

(4) ニュートン法を用いて、$\sqrt{2}$の値を求めよ（つまり$x^2 = 2$を考える）。

(5) $e^x = 3 x $の解を求めよ。

(6) $x^5 + x - 3 = 0$の解を求めよ。

(7) $f(x) = x + 3 \sin(x)$についてグラフを描いて概形を確認せよ。方程式$f(x)=0$に対して、初期値を(a) $x_0 = 0.1$、(b) $x_0 = 3$と取ってニュートン法を実行せよ。何が起こるだろうか。

## 二分法

![二分法](images/bisect.png)

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

図2. 二分法

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

いま、ある$x$の値$x_1$と$x_2$に対し、$f(x_1)$と$f(x_2)$は同符号でないことが分かっているとする。すると、関数$f(x)$が区間$[x_1, x_2]$で連続であれば、中間値の定理によりこの区間内に$f(x) = 0$の解が少なくとも1つは存在する。ここで$x_1$と$x_2$の中点、

<!-- textlint-enable -->

$$
x_\text{mid} = \frac{x_1 + x_2}{2} ,
\tag{7}
$$

を考える。もし$f(x_\text{mid})$が$f(x_1)$と同符号であれば、区間$[x_\text{mid}, x_2]$に少なくとも1つの解がある。もし$f(x_\text{mid})$が$f(x_2)$と同符号であれば、区間$[x_1, x_\text{mid}]$に少なくとも1つの解がある。このようにして、解の存在する区間を半分にできる。これを繰り返すと、いくらでも解の存在する区間を小さくでき、必要とする数値精度で解の値が求められる。これを二分法（bisection method）と呼ぶ。

In [ ]:
def bisect(f, x1, x2, n=100, verbose=False):
    # 方程式f(x)=0の、x1からx2の間にある1つの解を二分探索によって返す関数。
    # ただし、f(x1)とf(x2)は同符号ではないとする。
    #
    # n: 二分探索の分割数
    # verbose: Trueなら途中の収束状況を表示

    # まずはy1 = f(x1)とy2 = f(x2)を計算する。
    y1 = f(x1)
    y2 = f(x2)

    # f(x1)とf(x2)は同符号で合ってはならない。
    if y1 * y2 > 0:
        print("エラー: f(x1)とf(x2)が同符号")
        return  # 注意：エラーの時は値を何も返さない。

    # x1 <= x <= x2の間の解を二分探索で探す。
    for i in range(n):
        if verbose:
            print(f"i={i}, x1={x1}, x2={x2}")

        # 中間での関数の値を求める。
        xmid = (x1 + x2) / 2
        ymid = f(xmid)

        # 求めた値がy1とy2のどちらと同符号かを調べ、
        # その結果によって探索範囲を狭める。
        if ymid * y1 > 0:
            x1 = xmid
            y1 = ymid
        else:
            x2 = xmid
            y2 = ymid

    # x1とx2の中間値を近似解として返す。
    return (x1 + x2) / 2

上で定義した`bisect`関数を$x^2 - x - 6 = 0$に対して用いてみよう。注意として、求めたい解を挟んで、$x_1$と$x_2$を$f(x_1)$と$f(x_2)$が異符号となるように選ぶ必要がある。

In [ ]:
bisect(lambda x: x**2 - x - 6, -5, 0)

In [ ]:
bisect(lambda x: x**2 - x - 6, 0, 5)

二分法は$f(x)$が連続でさえあれば使うことができる。しかし、解を挟むような2点を与える必要があり、重解の場合は（解を挟んで$f(x)$が同符号となって）使えない。また、複素関数や多変数関数の求解へと一般化することも困難である。

## 練習問題

(8) 二分法を使って(a) $x^2 = 2$、(b) $e^x = 3 x$、(c) $x^5 + x - 3 = 0$、(d) $x + 3 \sin(x) = 0$の解を求めよ。

(9) 「彼（敵）を知り己を知れば百戦殆うからず」と言うように、戦いに勝つには「己」を知る必要がある。ニュートン法と二分法のそれぞれの長所と短所を比較してまとめよ<a name="cite_ref-5"></a>[<sup>[5]</sup>](#cite_note-5)。

## 演習課題

上の例題で定義された`newton`と`bisect`について、**そのまま変更せずに使うのなら**、解答セルにそのコピーを書かなくてもよいこととする。`make_plot`についても同様とする（解答にグラフを付ける必要は必ずしもないが、解のおおまかな値を検討するのに有用であろう）。以下の問題ではニュートン法と二分法のどちらを用いてもよい。

(1) 次の方程式の、$x=0$**以外**の解のうち1つを求めよ。

$$
\sin(x) = \frac{x}{2} .
\tag{8}
$$

In [ ]:
# 課題解答12.1  <-- 提出する際に、この行を必ず含めること。

from math import cos, sin

(2) 次の方程式の、$x = 0$**以外**の解を求めよ。この式は、プランクの法則からウィーンの変位則を導くときに現れる<a name="cite_ref-6"></a>[<sup>[6]</sup>](#cite_note-6)。

$$
(x - 5)  e^{x} + 5 = 0 .
\tag{9}
$$

In [ ]:
# 課題解答12.2  <-- 提出する際に、この行を必ず含めること。

from math import exp

(3) 次の方程式の正の解（$x > 0$）を、$a = 1$の場合について求めよ。ただし、平方根の中が正であるという条件によって$x$の範囲が制限されることに注意せよ。また、$x = 0$では分母が$0$となり、ゼロ除算によるエラーが起こることにも注意が必要である。この式は、量子力学で有限の高さの井戸型ポテンシャルでのエネルギー固有値を計算するときに現れる<a name="cite_ref-7"></a>[<sup>[7]</sup>](#cite_note-7)。

$$
\tan(x) = \frac{\sqrt{a-x^2}}{x} .
\tag{10}
$$

In [ ]:
# 課題解答12.3  <-- 提出する際に、この行を必ず含めること。

from math import cos, sqrt, tan

## 脚注

<!-- textlint-disable ja-technical-writing/no-doubled-joshi -->

<a name="cite_note-1"></a>1.&nbsp;[^](#cite_ref-1)
ところで、「方程式」は英語では「equation」と訳されるが、英語の「equation」は「等式」を意味する言葉である。つまり、変数を含んでいても含まなくても両辺が等しければ「equation」と呼ばれる。「等式」を表す単語の意味の言語による細かな違いは、[これ](https://www.academia.edu/3287674/What_is_an_Equation)の図6が分かりやすい。

<!-- textlint-enable -->

<a name="cite_note-2"></a>2.&nbsp;[^](#cite_ref-2)
式(3)のような連立1次方程式の求解は、線形代数学における一大テーマである。一般の場合に、いかに数値的に安定して高速に解を求めるかは厄介な問題であるが、ここでは取り扱わない。理工学の応用上の多くの問題は（場合によっては大規模な）線形方程式系に帰着する。

<!-- textlint-disable ja-technical-writing/max-kanji-continuous-len,ja-engineering-paper/prh -->

<a name="cite_note-3"></a>3.&nbsp;[^](#cite_ref-3)
簡単のために反復回数を固定しているが、実際は十分に収束したことが分かった時点で処理を抜けるようにしたほうがよいだろう。その場合、引数によって最大反復回数と許容誤差を与え、(a)1ステップでの変化が許容誤差以下になったら反復をやめる(b)最大反復回数を超えたら収束しないと考えエラーとする、などの処理を行うことになる。また、$f'(x)$が非常に$0$に近いときのエラー処理も必要であろう。

<!-- textlint-enable -->

<a name="cite_note-4"></a>4.&nbsp;[^](#cite_ref-4)
導関数が簡単に求まらない場合のアルゴリズムには、差分によって導関数を近似する割線法（secant method）や、その一般化である準ニュートン法（quasi-Newton method）などが知られている。

<!-- textlint-disable ja-engineering-paper/prh,ja-technical-writing/ja-no-weak-phrase -->

<a name="cite_note-5"></a>5.&nbsp;[^](#cite_ref-5)
「ニュートン法と二分法に長所と短所のあることは分かったが、両方のいいとこ取りはできないだろうか」と思うかもしれない。実際、そのようなアルゴリズムは存在しており、よく使われるのは割線法と二分法と逆2次補間を組み合わせたブレント法（Brent's method）である。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

<a name="cite_note-6"></a>6.&nbsp;[^](#cite_ref-6)
黒体放射の強度のピーク波長を求める。プランクの公式、

<!-- textlint-enable -->

$$
B(\lambda,T) d\lambda d\Omega = \frac{2 h c^2}{\lambda^5} \frac{1}{\exp\left(\frac{hc}{\lambda k_B T}\right) - 1}  d\lambda d\Omega,
$$

より、$B(\lambda,T)$を$\lambda$について偏微分したとき、これが$0$となる必要がある。$x=hc/(\lambda k_B T)$とおくと式<!-- eqref -->(9)が得られる。

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

<a name="cite_note-7"></a>7.&nbsp;[^](#cite_ref-7)
時間に依存しない1次元シュレディンガー方程式、

<!-- textlint-enable -->

$$
\left[ - \frac{\hbar^2}{2m} \frac{d^2}{dx^2} + V(x) \right] \psi(x) = E \psi(x) ,
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

において、いま（原点に対して対称な）有限の高さ（$V_0>0$）の井戸型ポテンシャル、

<!-- textlint-enable -->

$$
V(x) =
\begin{cases}
  0 , & |x| \le L , \\
  V_0 , & |x| > L ,
\end{cases}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-technical-writing/max-ten -->

に対する束縛状態（$E < V_0$）を考える。ポテンシャルが偶関数なので、波動関数$\psi(x)$は偶関数か奇関数であり、$x \ge 0$のみを考えれば十分である。$\psi(x)$が偶関数の場合、無限遠で$\psi \to 0$であるのと$x=L$での対数微分（$\psi'/\psi$）の連続性より、

<!-- textlint-enable -->

$$
k \tan(kL) = \kappa , \qquad
k \equiv \frac{\sqrt{2 m E}}{\hbar} , \qquad
\kappa \equiv \frac{\sqrt{2 m (V_0 - E)}}{\hbar} = \sqrt{\frac{2 m V_0}{\hbar^2} - k^2} ,
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

という条件が得られる（奇関数の場合は、$- k \cot(kL) = \kappa$という条件となる）。ここで$\xi = kL$とおくと、

<!-- textlint-enable -->

$$
\tan(\xi) = \frac{\sqrt{\frac{2mV_0L^2}{\hbar^2} - \xi^2}}{\xi} ,
$$

となり、これを満たす$\xi$を求めれば、エネルギー固有値$E=\hbar^2 \xi^2/(2 m L^2)$を求めることができる（教科書によっては作図による解の求め方が書いてある）。解の個数は$2mV_0L^2/\hbar^2$の値に依存する。